In [2]:
#install packages not native to Colabs
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.5MB 9.2MB/s 
     |████████████████████████████████| 901kB 45.1MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 1.2MB 8.3MB/s 


In [2]:
#%tensorflow_version 2.x
#import tensorflow as tf
import transformers
import numpy as np
import pandas as pd
import sklearn
import csv
import nltk
import random
import torch
import re
random.seed(1234) 
np.random.seed(1234)
torch.manual_seed(1234) 
#tf.random.set_seed(1234)
#from transformers import AdamW, get_linear_schedule_with_warmup
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
#from textwrap import wrap
#from pylab import rcParams

from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

AttributeError: type object 'h5py.h5.H5PYConfig' has no attribute '__reduce_cython__'

In [3]:
'''# Reading data using pandas located in same directory as the notebook
traindf = pd.read_csv('/content/drive/MyDrive/BERT Model + Data/train.csv', header=0, names=["Text","Label"])
traindf['Label'].unique()
index=traindf.index[traindf['Label']=='Label']
traindf.loc[index]
traindf.drop(index=index, inplace=True)
traindf['Text'].isnull
traindf.drop_duplicates
traindf.dropna(inplace=True)
traindf["Label"] = pd.to_numeric(traindf["Label"])
traindf, validdf = np.split(traindf, [int(.8*len(traindf))])

testdf = pd.read_csv('/content/drive/MyDrive/BERT Model + Data/test.csv', header=0, names=["Label", "Text"])
testdf['Label'].unique()
index=testdf.index[testdf['Label']=='Label']
testdf.loc[index]
testdf.drop(index=index, inplace=True)
testdf['Text'].isnull
testdf.dropna(inplace=True)
testdf.drop_duplicates
testdf["Label"] = pd.to_numeric(testdf["Label"])
testdf=testdf[["Text","Label"]]'''

In [14]:
traindf = pd.read_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Train.csv', header=0, names=["Text","Label"])
validdf = pd.read_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Valid.csv', header=0, names=["Text","Label"])
testdf = pd.read_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Test.csv', header=0, names=["Text", "Label"])
#testdf=testdf[["Text","Label"]]

In [22]:
print(testdf["Label"].unique())
print(traindf["Label"].unique())
print(validdf["Label"].unique())
#print(testdf["Text"].dtype)
#print(traindf["Text"].dtype)
#print(validdf["Text"].dtype)
#print(traindf)
#print(validdf)
traindf.dropna(inplace=True)
print(traindf.index[traindf['Text'].isnull().values])
#print(traindf['Text'][5606])
#print(testdf)

[1 0]
[0 1]
[0 1]
Int64Index([], dtype='int64')


In [6]:
'''# Function to clean text. Remove stopwords, hyperlinks, emojis, etc..
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
 
traindf['Text'] = traindf['Text'].apply(clean_text)
validdf['Text'] = validdf['Text'].apply(clean_text)
testdf['Text'] = testdf['Text'].apply(clean_text)

from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
traindf['Text']=traindf['Text'].apply(lambda x:" ".join([i for i in x.split() if i not in stop]))
validdf['Text']=validdf['Text'].apply(lambda x:" ".join([i for i in x.split() if i not in stop]))
testdf['Text']=testdf['Text'].apply(lambda x:" ".join([i for i in x.split() if i not in stop]))

# List of class names.
class_names = [0, 1]

#output the pre-processed train_set
traindf.to_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Train.csv', index=False)
validdf.to_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Valid.csv', index=False)
testdf.to_csv('/content/drive/MyDrive/BERT Model + Data/Preprocessed_Test.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
#Function for encoding data
class TrainDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding=True,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [24]:
#Function for DataLoader
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = TrainDataset(
    reviews=df.Text.to_numpy(),
    targets=df.Label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    drop_last=True
  )

In [25]:
'''#Import tokenizer and set the Max Length after tokenization
from transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

MAX_LEN = 0

# For every sentence...
for sent in pd.concat([traindf['Text'],validdf['Text']],ignore_index=True):
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    # Update the maximum sentence length.
    MAX_LEN = max(MAX_LEN, len(input_ids))

MAX_LEN=min(MAX_LEN,512)
print('Max sentence length: ', MAX_LEN)

KeyboardInterrupt: ignored

In [26]:
#Import tokenizer and set the Max Length after tokenization
from transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

#Process the data from train, validation, test
MAX_LEN=512
BATCH_SIZE = 8
train_data_loader = create_data_loader(traindf, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(validdf, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(testdf, tokenizer, MAX_LEN, BATCH_SIZE)

In [27]:
#import the model and save to GPU/CPU
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

#Can reduce Epochs if insufficient memory
EPOCHS = 6
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [28]:
from sklearn import metrics

#Create the training function and display the results
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(8,MAX_LEN).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [29]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(8,MAX_LEN).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [30]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(traindf)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(validdf)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        #torch.save(model.state_dict(), '/content/drive/MyDrive/Assignment 2/Models.py')
        best_accuracy = val_acc

Epoch 1/6
----------
Train loss 0.07674663383088304 Train accuracy 0.9829917002012073
Val loss 0.05011604751601791 Val accuracy 0.9918259557344065

Epoch 2/6
----------
Train loss 0.04791015869326294 Train accuracy 0.9907727615694165
Val loss 0.043779299759261626 Val accuracy 0.9922660965794768

Epoch 3/6
----------
Train loss 0.029268079510404433 Train accuracy 0.994765467806841
Val loss 0.03879204812552376 Val accuracy 0.9927062374245473

Epoch 4/6
----------
Train loss 0.019652264831652437 Train accuracy 0.9964474346076458
Val loss 0.030944621664859232 Val accuracy 0.9954728370221329

Epoch 5/6
----------
Train loss 0.012912695901516067 Train accuracy 0.99809796277666
Val loss 0.01893211367380203 Val accuracy 0.9966046277665996

Epoch 6/6
----------
Train loss 0.00586679065902806 Train accuracy 0.9990568410462777
Val loss 0.018064424358497935 Val accuracy 0.9964159959758552

CPU times: user 11h 12min 45s, sys: 3min 45s, total: 11h 16min 30s
Wall time: 11h 5min 21s


In [31]:
#Save parameters for future retrieval
torch.save(model.state_dict(), '/content/drive/MyDrive/BERT Model + Data/Modelparams.pt')

In [32]:
class_names = [0, 1] 
#Create function to formulate prediction on test set
def predict_sentiment(text):
    review_text = text

    encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoded_review['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_review['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,MAX_LEN).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)

    #print("Positive score:", probs[1])
    #print("Negative score:", probs[0])
    #print(f'Review text: {review_text}')
    return class_names[prediction]

In [33]:
#Place all testset through the above function
testlabels=[]
for text in testdf.Text.tolist():
  testlabels.append(predict_sentiment(text))

testdf['Label']=testlabels
print(testdf)

#Save the results to a new csv called 'results.csv'
testdf.to_csv('/content/drive/MyDrive/BERT Model + Data/results.csv', index=False)

NameError: ignored